IMPORT PACKAGES

In [1]:
import pandas as pd
import json as js
from tkinter import *
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os
import pymongo
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

INITIALIZE MONGO SERVER

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

INITIALIZE VARIABLES

In [3]:
db = client['university_logs']
activity_mongo = db['activity_log']
component_mongo = db['component_codes']
user_mongo = db['user_log']
filepath = ''
json_file = ''
activity_log_loaded = False
component_codes_loaded = False
user_log_loaded = False
row = 0
file_frames = {}
root = tk.Tk()
root.geometry="1180x820"
root.resizable(True,True)
root.title("Summative Assessment")
root.configure(bg='#3e3e42')
style = ttk.Style()


DECLARE GLOBAL VARIABLES

In [4]:
global db
global treeview
global user_log, activity_log, component_codes,user_activity,activity_count_reset, filtered
global save_cc,save_aclog,save_userlog, mongo_aclog,mongo_cc,mongo_userlog
global button_save,button_open,button_clean,button_mongo,button_cleanup,\
    button_merge,button_pivot,button_multi, button_statistics, button_open_graphs
global choice,var,var2,choice2
global ac_frame,cc_frame,ul_frame,in_frame,here_frame,merge_frame,stats_frame,final_merged_frame,graphs_frame,graphs_in_frame
global activity_text,component_text,user_text
global activity_frame,component_frame,user_frame
global activity_ins,component_ins,user_ins

FUNCTION TO OPEN FILES


In [5]:
def open_file():
    global activity_log, component_codes, user_log, db, button_save, save_userlog, save_cc, save_aclog, button_open, button_clean, activity_log_loaded, component_codes_loaded, user_log_loaded, choice, ac_frame, cc_frame, ul_frame, row, file_frames, button_mongo,mongo_aclog,mongo_cc,mongo_userlog,activity_ins,component_ins,user_ins

    # Create file dialog window using tkinter.filedialog module
    file = filedialog.askopenfile(mode='r', filetypes=[('CSV Files', '*.csv')])
    if file:
        # GENERATE FILE PATH AND INITIALIZE PANDAS DATAFRAME
        filepath = os.path.abspath(file.name)
        db = pd.read_csv(filepath)
        # CHECK IF FILE STRUCTURE CORRESPONDS TO ACTIVITY_LOG.csv
        if {'User Full Name *Anonymized','Component','Action','Target'}.issubset(db.columns):
            # IF ACTIVITY LOG IS ALREADY LOADED, ASK TO DELETE IT FIRST
            if activity_log_loaded:
                messagebox.showwarning("Error", "You have already loaded an ACTIVITY_LOG.csv file. Please Delete it before opening another one")
                return None
            # ELSE LOAD FILE INTO DATAFRAME, CREATE TABLE VIEW AND MANAGE BUTTONS
            else:
                activity_log = db
                messagebox.showinfo("Success", "ACTIVITY_LOG.csv is loaded successfully")
                ac_frame = create_file_frame(1,'activity_log')
                label=ttk.Label(ac_frame, text='Activity Log:', foreground='black',background='#FFC0CB')
                label.pack(expand=False,anchor='w',padx=30)
                load_table_view(activity_log, ac_frame,1)

                delete = ttk.Button(ac_frame, text="Delete", command=lambda: delete_file(1))
                delete.pack(expand=False,anchor='ne',padx=30)
                button_save['state'] = 'normal'
                save_aclog['state'] = 'normal'
                button_open['state'] = 'normal'
                button_mongo['state'] = 'normal'
                save_menu.entryconfig(0, state='normal')
                backup_menu.entryconfig(0, state='normal')


                mongo_aclog['state'] = 'normal'
                activity_log_loaded = True
        # SAME AS ABOVE
        elif {"Component",'Code'}.issubset(db.columns):
            if component_codes_loaded:
                messagebox.showwarning("Error", "You have already loaded a COMPONENT_CODES.csv file. Please Delete it before opening another one")
                return None
            else:
                component_codes = db
                messagebox.showinfo("Success", "COMPONENT_CODES.csv is loaded successfully")
                cc_frame = create_file_frame(2,'component_codes')
                label=ttk.Label(cc_frame, text='Component Codes:', foreground='black',background='#FFC0CB')
                label.pack(expand=False,anchor='w',padx=30)
                load_table_view(component_codes, cc_frame,2)
                delete = ttk.Button(cc_frame, text="Delete", command=lambda: delete_file(2))
                delete.pack(expand=False,anchor='ne',padx=30)
                save_cc['state'] = 'normal'
                mongo_cc['state'] = 'normal'
                save_menu.entryconfig(1, state='normal')
                backup_menu.entryconfig(1, state='normal')

                button_save['state'] = 'normal'
                button_open['state'] = 'normal'
                button_mongo['state'] = 'normal'
                component_codes_loaded = True
        # SAME AS ABOVE
        elif {'Date','Time','User Full Name *Anonymized'}.issubset(db.columns):
           if user_log_loaded:
                messagebox.showwarning("Error", "You have already loaded an USER_LOG.csv file. Please Delete it before opening another one")
                return None
           else:

                user_log = db[['User Full Name *Anonymized','Date','Time']]
                messagebox.showinfo("Success", "USER_LOG.csv is loaded successfully")
                ul_frame = create_file_frame(3,'user_log')
                label=ttk.Label(ul_frame, text='User Log:', foreground='black',background='#FFC0CB')
                label.pack(expand=False,anchor='w',padx=30)
                load_table_view(user_log, ul_frame,3)
                delete = ttk.Button(ul_frame, text="Delete", command=lambda: delete_file(3))
                delete.pack(expand=False,anchor='e',padx=30)
                backup_menu.entryconfig(2, state='normal')
                save_menu.entryconfig(2, state='normal')
                save_userlog['state'] = 'normal'
                mongo_userlog['state'] = 'normal'
                button_save['state'] = 'normal'
                button_open['state'] = 'normal'
                button_mongo['state'] = 'normal'
                user_log_loaded = True
        else:
            # SHOW WARNING IF FILE STRUCTURE DOESN'T MATCH
            messagebox.showerror("Error", "The structure of your file doesn't match the requirements for ACTVITY_LOG.csv Please check the file and try again.")
        # IF BOTH ACTIVITY AND USER LOG ARE LOADED, ENABLE THE CLEAN UP TAB
        if activity_log_loaded and user_log_loaded:
            button_clean['state'] = 'normal'
            filemenu.entryconfig(3, state='normal')
            dwmenu.entryconfig(0, state='normal')
            wrangling_tab()
        reposition_frames()



CREATE FILE MANAGEMENT TAB


In [6]:
def file_managent_tab():

    global save_aclog,save_cc,save_userlog,button_save,button_open,button_clean,activity_log,component_codes,user_log,choice,choice2,var,var2,activity_log_loaded,component_codes_loaded,user_log_loaded,ac_frame,cc_frame,ul_frame,in_frame,sheets_frame,row,button_mongo,mongo_aclog,mongo_cc,mongo_userlog,activity_ins,component_ins,user_log, activity_log,user_log

    var = IntVar()
    var2 = IntVar()
    choice = 0
    choice2 = 0

    # CREATE TKINTER STYLES FOR FILE MANAGEMENT TAB
    style.configure('TLabelframe.Label',foreground='#3e3e42',background='#FFC0CB')
    style.configure('TRadiobutton',foreground='#3e3e42',background='#DCDAD4',highlightbackground='#FFC0CB',bg='#FFC0CB')
    style.configure('TFrame',background='#FFC0CB',border=NONE)
    style.configure('Frame1.TFrame',background='#FFC0CB',borderwidth=1)
    style.configure('Button2.TButton',background='#F1CD89')

    # CREATE INTERNAL FRAME FOR FILE MANAGEMENT TAB
    in_frame = ttk.Frame(file_management,style='Frame1.TFrame')

    # CONFIGURE GRID OF INTERNAL FRAME
    in_frame.grid_configure()
    in_frame.columnconfigure(0, minsize=100)
    in_frame.columnconfigure(1, weight=1)
    in_frame.pack(expand=True, fill=BOTH)

    style.configure('Frame2.TFrame',background='#DCDAD4',border=NONE)

    # CREATE FRAME TO HOST BUTTONS
    next_frame = ttk.Frame(in_frame, width=1000, height=100, borderwidth=4,style='Frame2.TFrame')
    # CREATE BUTTON OPENING CSV FILE
    button_open = ttk.Button(next_frame, text="Open CSV File", command=lambda: open_file())
    button_open.grid(column=0, row=0, padx=10, pady=10,sticky='w')

    # CREATE BUTTON TO LOAD WRANGLING TAB
    button_clean = ttk.Button(next_frame, text="Clean Up Files", command=lambda: notebook_select(1),state=DISABLED,style='Button2.TButton')
    button_clean.grid(row=1, column=0, padx=10, pady=10, sticky='w')

    # CREATE RADIO BUTTONS FOR FILE SELECTION
    save_aclog = ttk.Radiobutton(next_frame, text='ACTIVITY_LOG', variable=var, value=1,state=DISABLED,command=lambda: get_choice(1))
    save_aclog.grid(column=0, row=3,padx=10, sticky='w')
    save_cc = ttk.Radiobutton(next_frame, text='COMPONENT_CODES', variable=var, value=2,state=DISABLED,command=lambda: get_choice(1))
    save_cc.grid(column=0, row=4,padx=10, sticky='w')
    save_userlog = ttk.Radiobutton(next_frame, text='USER_LOG', variable=var, value=3,state=DISABLED,command=lambda: get_choice(1))
    save_userlog.grid(column=0, row=5,padx=10,pady=(0,10),sticky='w')

    # CREATE SAVE BUTTON
    button_save = ttk.Button(next_frame, text="Save JSON file: ",state=DISABLED, command=lambda: save_file(choice))
    button_save.grid(row=2, column=0, padx=10, pady=10, sticky='w')


    # CREATE RADIO BUTTONS FOR FILE SELECTION
    mongo_aclog = ttk.Radiobutton(next_frame, text='ACTIVITY_LOG', variable=var2, value=1, state=DISABLED, command=lambda: get_choice(2))
    mongo_aclog.grid(column=0, row=7,padx=10, sticky='w')
    mongo_cc = ttk.Radiobutton(next_frame, text='COMPONENT_CODES', variable=var2, value=2, state=DISABLED, command=lambda: get_choice(2))
    mongo_cc.grid(column=0, row=8,padx=10, sticky='w')
    mongo_userlog = ttk.Radiobutton(next_frame, text='USER_LOG', variable=var2, value=3, state=DISABLED, command=lambda: get_choice(2))
    mongo_userlog.grid(column=0, row=9,padx=10,pady=(0,10),sticky='w')

    # CREATE SAVE TO MONGO BUTTON
    button_mongo = ttk.Button(next_frame, text="Backup to Mongo Server: ",state=DISABLED, command=lambda: save_to_mongo(choice2))
    button_mongo.grid(row=6, column=0, padx=10, pady=10, sticky='w')

    # PLACE FRAME IN MASTER
    next_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # IF ALL 3 FILES LOADED, ENABLE BUTTON CLEAN UP
    if activity_log_loaded and component_codes_loaded and user_log_loaded:
        button_clean['state'] = 'normal'



SAVE DATABASE TO MONGODB

In [7]:
def save_to_mongo(file):
    global activity_ins,component_ins,user_ins

    # CAST FILE NUMBER AS INT
    file = int(file)
    # IF FILE = 1, SAVE ACTIVITY_LOG TO MONGO
    if file == 1:
        # ERROR MANAGEMENT
        try:
            activity_ins = activity_mongo.insert_many(activity_log.to_dict('records'))
            messagebox.showinfo('Success!','The file was successfully saved to Mongo Server.')
            print(activity_ins.inserted_ids)
        except Exception as e:
            messagebox.showerror("Error", "An error occurred while saving to Mongo Server. Please check your connection and try again.")
            print(e)
    # IF FILE = 2, SAVE COMPONENT_CODES TO MONGO
    elif file == 2:
    # ERROR MANAGEMENT
        try:
            component_ins = component_mongo.insert_many(component_codes.to_dict('records'))
            messagebox.showinfo('Success!','The file was successfully saved to Mongo Server.')
            print(component_ins.inserted_ids)
        except Exception as e:
            messagebox.showerror("Error", "An error occurred while saving to Mongo Server. Please check your connection and try again.")
            print(e)
    # IF FILE = 3, SAVE USER_LOG TO MONGO
    elif file == 3:
        try:
            user_ins = user_mongo.insert_many(user_log.to_dict('records'))
            messagebox.showinfo('Success!','The file was successfully saved to Mongo Server.')
            print(user_ins.inserted_ids)
        except Exception as e:
            messagebox.showerror("Error", "An error occurred while saving to Mongo Server. Please check your connection and try again.")
            print(e)
    else:
        messagebox.showerror("Error","Please select a dataset to save")
    button_mongo['state'] = 'normal'

REPLACE DELETED TABLE WITH LOADED ONE

In [8]:
def create_file_frame(file_type,frame_key):
    global in_frame, file_frames
    new_frame = ttk.Frame(in_frame,borderwidth=0,relief='flat',padding=10)
    file_frames[frame_key] = new_frame
    reposition_frames()
    return new_frame

LOGIC TO DELETE FILE AND MOVE ANOTHER TO THE TOP

In [9]:
def delete_file(file):
    global ac_frame,cc_frame,ul_frame,activity_log_loaded,component_codes_loaded,user_log_loaded,activity_log,component_codes,user_log,file_frames
    frame_key = ''
    if messagebox.askyesno('Warning', 'Are you sure you want to delete this dataset?'):
        if file == 1:
            frame_key = 'activity_log'
            activity_log_loaded = False
            activity_log = None
            save_aclog['state'] = 'disabled'
        elif file == 2:
            frame_key = 'component_codes'
            component_codes_loaded = False
            component_codes = None
            save_cc['state'] = 'disabled'
        elif file == 3:
            frame_key = 'user_log'
            user_log_loaded = False
            user_log = None
            save_userlog['state'] = 'disabled'

        if frame_key in file_frames:
            file_frames[frame_key].grid_forget()
            del file_frames[frame_key]
        reposition_frames()

    else:
        return None


REPOSITION FRAMES FOR TREEVIEW

In [10]:
def reposition_frames():
    global file_frames
    row = 0
    for frame_key, frame in file_frames.items():
        frame.grid(row=row, column=1)
        row+=1

GET CHOICE FROM RADIO BUTTONS

In [11]:
# GET CHOSEN RADIO BUTTON AND SEND VALUE TO BUTTON
def get_choice(which):
    global choice,var,choice2,var2
    if which == 1:
        choice = str(var.get())
    if which == 2:
        choice2 = str(var2.get())


SELECT A NOTEBOOK TAB

In [12]:
def notebook_select(nb):
    notebook.select(nb)


CREATE WRANGLING TAB

In [13]:

def wrangling_tab():
    notebook.tab(1, state='normal')

    global activity_text,component_text,user_text,activity_frame,component_frame,user_frame,activity_log,user_log,button_cleanup,button_merge,button_pivot,button_multi,here_frame,merge_frame, button_statistics,button_open_graphs, button_multi

    # CONFIGURE STYLES FOR WRANGLING TAB
    style.configure('Menu.TFrame',background='#DCDAD4',highlightcolor='#DCDAD4',)
    style.configure('LabelFrame.Label',background='#DCDAD4')
    style.configure('Frame3.TFrame',background='#F1CDB0',border=NONE)
    style.configure('Button.TButton',background='#F1CD89',border=NONE)
    # CREATE INTERNAL FRAME FOR WRANGLING TAB
    here_frame = ttk.Frame(dataset_management)
    here_frame.configure(style='Frame3.TFrame')
    # CONFIGURE COLUMNS
    here_frame.columnconfigure(0, minsize=100)
    here_frame.columnconfigure(1, weight=1)
    #PACK INTERNAL FRAME
    here_frame.pack(expand=True, fill=BOTH)
    # CREATE FRAME FOR MENU
    menu_frame = ttk.Frame(here_frame, style="Menu.TFrame")
    menu_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')
    # CREATE CLEAN UP BUTTON
    cleanup_labelwidget = ttk.Label(menu_frame, text='Initial Clean Up \n'
                                                     '(Remove NAs, Fix Data Types, \n'
                                                     'Fix Dates: ',background='#DCDAD4')
    cleanup_frame = ttk.LabelFrame(menu_frame,labelwidget=cleanup_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_cleanup = ttk.Button(cleanup_frame, text='OK', command=lambda: cleanup(),style='Button.TButton')
    button_cleanup.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    cleanup_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # CREATE MERGE BUTTON
    merge_labelwidget = ttk.Label(menu_frame, text='Merge ACTIVITY_LOG and USER_LOG\n'
                                                   'Rename column "User Full Name"\n'
                                                   'Remove entries for\n'
                                                   'System and Folder: ',background='#DCDAD4')
    mergemenu_frame = ttk.LabelFrame(menu_frame,labelwidget=merge_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_merge = ttk.Button(mergemenu_frame, text='OK',state='disabled', command=lambda: merge(),style='Button.TButton')
    button_merge.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    mergemenu_frame.grid(column=0, row=1, padx=10, pady=10,sticky='NW')

    # CREATE ACTIVITY COUNT BUTTON
    multi_labelwidget = ttk.Label(menu_frame, text='Create Multiindex with\n'
                                                   'Activity Count: ',background='#DCDAD4')
    multi_frame = ttk.LabelFrame(menu_frame,labelwidget=multi_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_multi = ttk.Button(multi_frame, text='OK',state='disabled', command=lambda: activity_count(),style='Button.TButton')
    button_multi.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    multi_frame.grid(column=0, row=2, padx=10, pady=10,sticky='NW')

    # CREATE OPEN STATISTICS BUTTON
    statistics_labelwidget = ttk.Label(menu_frame, text='Open Statistics Tab: ',background='#DCDAD4')
    statistics_button_frame = ttk.LabelFrame(menu_frame,labelwidget=statistics_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_statistics = ttk.Button(statistics_button_frame, text='OK',state='disabled', command=lambda: notebook_select(2),style='Button2.TButton')
    button_statistics.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    statistics_button_frame.grid(column=0, row=3, padx=10, pady=10,sticky='NW')

    # CREATE OPEN GRAPHS BUTTON
    open_graphs_tab_labelwidget = ttk.Label(menu_frame, text='Open Graphs Tab: ',background='#DCDAD4')
    open_graphs_tab_frame = ttk.LabelFrame(menu_frame,labelwidget=open_graphs_tab_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_open_graphs = ttk.Button(open_graphs_tab_frame, text='OK',state = 'disabled', command=lambda: notebook_select(3),style='Button3.TButton')
    button_open_graphs.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    open_graphs_tab_frame.grid(column=0, row=4, padx=10, pady=10,sticky='NW')

    # CREATE FRAME TO HOST TABLES
    activity_frame = Frame(here_frame, bg='#F1CDB0')
    activity_text = ttk.Label(activity_frame, text='Activity Log:', foreground='black',background='#F1CDB0')
    activity_text.pack(expand=False,fill='none')
    activity_frame.grid(column=1, row=0, padx=10, pady=10,sticky='N')


    # CREATE TABLES
    load_table_view(activity_log,activity_frame,1)
    user_text = ttk.Label(activity_frame, text='User Log:', foreground='black',background='#F1CDB0')
    user_text.pack(expand=False,fill='none')
    load_table_view(user_log, activity_frame,3)




CLEAN UP FUNCTION

In [14]:
def cleanup():
    global activity_text,component_text,user_text,activity_frame,component_frame,user_frame,activity_log,user_log,component_codes

    # DROP NAN VALUES
    activity_log.dropna(axis=0, how='any',inplace=True)
    # REPLACE ATTENDENCE WITH ATTENDANCE
    activity_log.replace('Attendence','Attendance',inplace=True)
    # SORT ACTIVITY LOG TABLE
    activity_log.sort_values(by=['User Full Name *Anonymized','Component'],ascending=True,inplace=True)
    # RENAME DATE COLUMN
    user_log['Year/Month'] = user_log['Date']
    # KEEP ONLY YEAR/MONTH
    user_log['Year/Month'] = pd.to_datetime(user_log['Year/Month'].str[:10],format='%d/%m/%Y').apply(lambda x: x.strftime('%m/%Y'))
    # KEEP ONLY DAY
    user_log['Day'] = pd.to_datetime(user_log['Date'].str[:10],format='%d/%m/%Y').apply(lambda x: x.strftime('%d'))
    # SORT USER LOG TABLE
    user_log.sort_values(by=['User Full Name *Anonymized','Year/Month','Day'],ascending=True,inplace=True)

    # REMOVE WIDGETS FROM ACTIVITY FRAME
    for widget in activity_frame.winfo_children():
        widget.destroy()
    # REPLACE WITH NEWLY CLEANED UP DATASETS
    activity_text = ttk.Label(activity_frame, text='Activity Log (Cleaned Up):', foreground='black',background='#F1CDB0')
    activity_text.pack()
    load_table_view(activity_log,activity_frame,4)
    user_text = ttk.Label(activity_frame, text='User Log (Cleaned Up):', foreground='black',background='#F1CDB0')
    user_text.pack()
    load_table_view(user_log,activity_frame,5)

    # DISABLE BUTTON CLEANUP AND ENABLE BUTTON MERGE
    button_cleanup['state'] = 'disabled'
    button_merge['state'] = 'normal'
    dwmenu.entryconfig(0,state='disabled')
    dwmenu.entryconfig(1,state='normal')

    # UPDDATE USER ON SUCCESS OF THE PROCESS
    messagebox.showinfo('Success','The datasets were successfully cleaned up.')

MERGE DATASETS FUNCTION

In [15]:
def merge():
    global user_activity, merge_frame

    # RENAME USER FULL NAME COLUMN TO USER_ID
    user_log.rename(columns={'User Full Name *Anonymized':'User_ID'},inplace=True)
    activity_log.rename(columns={'User Full Name *Anonymized':'User_ID_1'},inplace=True)

    # MERGE USER AND ACTIVITY LOG
    user_activity = pd.concat([user_log,activity_log],axis=1)

    # REMOVE ALL ENTRIES CONTAINING SYSTEM OR FOLDER
    user_activity = user_activity[user_activity['Component'] != 'System']
    user_activity = user_activity[user_activity['Component'] != 'Folder']

    # DROP DUPLICATE USER_ID_1 AND DATE COLUMNS
    user_activity.drop(['User_ID_1','Date'],axis=1,inplace=True)

    # CREATE INDEX LIST FROM COLUMNS
    indexes = [user_activity['User_ID'],user_activity['Year/Month'],user_activity['Component'],user_activity['Day']]
    # CREATE MULTIINDEX FROM INDEX LIST
    multiindex = pd.MultiIndex.from_arrays(indexes,names=('User_ID','Year/Month','Component','Day'))
    # REMOVE ORIGINAL COLUMNS AND KEEP MULTIINDEX
    user_activity.drop(['User_ID','Year/Month','Day','Component'],axis=1,inplace=True)
    user_activity.index = multiindex
    # SORT BY USER_ID, THEN YEAR/MONTH, THEN COMPONENT
    user_activity.sort_values(['User_ID','Year/Month','Component'],inplace=True)

    # CREATE COPY OF MERGED DB WITHOUT INDEX
    merged_activity = user_activity.reset_index(inplace=False)

    # PLACE NEW MERGED DB IN TAB
    activity_frame.grid(column=1, row=1, padx=10, pady=10,sticky='NE')
    merge_frame = Frame(here_frame,bg='#F1CDB0')
    merge_frame.grid(column=1, row=0, padx=10, pady=10,sticky='NE')

    merge_text = ttk.Label(merge_frame, text='Merged DBs:', foreground='black',background='#F1CDB0')
    merge_text.pack(expand=False,fill='none')
    load_table_view(merged_activity,merge_frame,5)

    button_merge['state'] = 'disabled'
    button_multi['state'] = 'normal'
    dwmenu.entryconfig(1,state='disabled')
    dwmenu.entryconfig(2,state='normal')

    messagebox.showinfo('Success','The datasets were successfully merged into a single DB and a multiindex was created.')


    return None

ADD ACTIVITY COUNT TO MERGED DB

In [16]:
def activity_count():
    global user_activity,merge_frame,activity_count_reset,button_multi,button_statistics,button_open_graphs

    # CREATE TOTAL ACTIVITY COUNT COLUMN BY GROUPING LEVELS 0,1,2
    user_activity['Activity Count'] = user_activity.groupby(level=[0,1,2]).size()

    # RESET INDEX OF TABLE
    user_activity.reset_index(inplace=True)

    # CREATE INDEX LIST FROM COLUMNS INCLUDING ACTIVITY COUNT
    indexes = [user_activity['User_ID'],user_activity['Year/Month'],user_activity['Component'],user_activity['Activity Count'],user_activity['Day']]
    # CREATE MULTIINDEX FROM INDEX LIST
    multiindex = pd.MultiIndex.from_arrays(indexes,names=('User_ID','Year/Month','Component','Activity Count','Day'))

    user_activity.index = multiindex

    # DROP DUPLICATE COLUMNS
    user_activity.drop(['User_ID','Year/Month','Component','Activity Count','Day'],axis=1,inplace=True)

    # CREATE NEW DB WITHOUT MULTIINDEX FROM LAST CREATED DB
    activity_count_reset = user_activity.reset_index(inplace=False)

    # REMOVE ALL WIDGETS FROM FRAME
    for widget in merge_frame.winfo_children():
        widget.destroy()

    # LOAD UPDATED MERGED DB WITH ACTIVITY COUNT INTO TAB
    merge_text = ttk.Label(merge_frame, text='Merged DBs With Activity Count:', foreground='black',background='#F1CDB0')
    merge_text.pack(expand=False,fill='none')
    load_table_view(activity_count_reset,merge_frame,6)

    # SHOW SUCCESS MESSAGE AND DISABLE/ENABLE BUTTONS
    messagebox.showinfo('Success','The Activity Count index was added to the multiindex.')

    button_multi['state'] = 'disabled'
    button_statistics['state'] = 'normal'
    button_open_graphs['state'] = 'normal'
    dwmenu.entryconfig(2,state='disabled')
    dwmenu.entryconfig(3,state='normal')
    dwmenu.entryconfig(4,state='normal')
    dwmenu.entryconfig(5,state='normal')
    for x in range(0,10):
        statsmenu.entryconfig(x,state='normal')
    for y in range(0,8):
        graphsmenu.entryconfig(y,state='normal')

    # LOAD STATISTICS AND GRAPHS TABS IN THE BACKGROUND
    statistics_tab()
    graphs_tab()




OPEN STATISTICS TAB

In [17]:
def statistics_tab():
    global activity_text,component_text,user_text,activity_frame,component_frame,user_frame,activity_log,user_log,button_cleanup,button_merge,button_pivot,button_multi,here_frame,merge_frame, button_statistics, final_merged_frame,stats_frame, db_for_statistics_reset, filtered

    # CREATE A COPY OF THE DB TO PROCESS STATISTICS
    db_for_statistics = activity_count_reset.copy()
    indexes = [db_for_statistics['User_ID'],db_for_statistics['Year/Month'],db_for_statistics['Component'],db_for_statistics['Activity Count'],db_for_statistics['Day']]
    multiindex = pd.MultiIndex.from_arrays(indexes,names=('User_ID','Year/Month','Component','Activity Count','Day'))


    db_for_statistics.index = multiindex
    # DROP UNNECESSARY COLUMNS FOR THE TASK
    db_for_statistics.drop(['User_ID','Year/Month','Component','Activity Count','Day'],axis=1,inplace=True)

    # FILTER COMPONENTS ACCORDING TO THE REQUIREMENTS
    filtered = db_for_statistics[user_activity.index.get_level_values('Component').isin(['Quiz', 'Lecture', 'Assignment', 'Attendance', 'Survey'])]
    filtered.reset_index(inplace=True)

    notebook.tab(2, state='normal')

    # CONFIGURE STYLES FOR STATISTICS TAB
    style.configure('Menu.TFrame',background='#DCDAD4',highlightcolor='#DCDAD4',)
    style.configure('LabelFrame.Label',background='#DCDAD4')
    style.configure('Frame4.TFrame',background='#89AEBD',border=NONE)
    style.configure('Button2.TButton',background='#89AEBD',border=NONE)

    # CREATE INTERNAL FRAME FOR STATISTICS TAB
    stats_frame = ttk.Frame(statistics)
    stats_frame.configure(style='Frame4.TFrame')
    stats_frame.columnconfigure(0, minsize=100)
    stats_frame.columnconfigure(1, weight=1)
    stats_frame.rowconfigure(0, minsize=100)
    stats_frame.rowconfigure(1, weight=1)
    stats_frame.pack(expand=True, fill=BOTH)

    # CREATE MENU FRAME TO HOST BUTTONS
    statsmenu_labelwidget = ttk.Label(stats_frame, text='Component Statistics:',background='#DCDAD4')
    statsmenu_frame = ttk.LabelFrame(stats_frame,labelwidget=statsmenu_labelwidget, style="Menu.TFrame")
    statsmenu_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # CREATE MODE PER MONTH BUTTON
    monthmode_labelwidget = ttk.Label(statsmenu_frame, text='Components Mode\n'
                                                            'by month (with pivoting):',background='#DCDAD4')
    monthmode_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=monthmode_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_monthmode = ttk.Button(monthmode_frame, text='OK', command=lambda: mode_per_month(),style='Button2.TButton')
    button_monthmode.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    monthmode_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # CREATE MODE PER 13 WEEKS BUTTON
    yearmode_labelwidget = ttk.Label(statsmenu_frame, text='Components Mode\n'
                                                            'for 13 weeks:',background='#DCDAD4')
    yearmode_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=yearmode_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_yearmnode = ttk.Button(yearmode_frame, text='OK', command=lambda: mode_per_13_weeks(),style='Button2.TButton')
    button_yearmnode.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    yearmode_frame.grid(column=0, row=1, padx=10, pady=10,sticky='NW')

    # CREATE MEDIAN PER MONTH BUTTON
    monthmedian_labelwidget = ttk.Label(statsmenu_frame, text='Components Median\n'
                                                              'by month (with pivoting): ',background='#DCDAD4')
    monthmedian_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=monthmedian_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_monthmedian = ttk.Button(monthmedian_frame, text='OK', command=lambda: median_per_month(),style='Button2.TButton')
    button_monthmedian.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    monthmedian_frame.grid(column=0, row=2, padx=10, pady=10,sticky='NW')

    # CREATE MEDIAN PER 13 WEEKS BUTTON
    yearmedian_labelwidget = ttk.Label(statsmenu_frame, text='Components Median\n'
                                                              'for 13 weeks: ',background='#DCDAD4')
    yearmedian_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=yearmedian_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_yearmedian = ttk.Button(yearmedian_frame, text='OK', command=lambda: median_per_13_weeks(),style='Button2.TButton')
    button_yearmedian.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    yearmedian_frame.grid(column=0, row=3, padx=10, pady=10,sticky='NW')

    # CREATE MEAN PER MONTH BUTTON
    monthmean_labelwidget = ttk.Label(statsmenu_frame, text='Components Mean\n'
                                                              'by month (with pivoting): ',background='#DCDAD4')
    monthmean_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=monthmean_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_monthmean = ttk.Button(monthmean_frame, text='OK', command=lambda: mean_per_month(),style='Button2.TButton')
    button_monthmean.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    monthmean_frame.grid(column=0, row=4, padx=10, pady=10,sticky='NW')

    # CREATE MEAN Per 13 Weeks BUTTON
    yearmean_labelwidget = ttk.Label(statsmenu_frame, text='Components Mean\n'
                                                              'for 13 weeks: ',background='#DCDAD4')
    yearmean_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=yearmean_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_yearmean = ttk.Button(yearmean_frame, text='OK', command=lambda: mean_per_13_weeks(),style='Button2.TButton')
    button_yearmean.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    yearmean_frame.grid(column=0, row=5, padx=10, pady=10,sticky='NW')

    # CREATE COUNT PER MONTH BUTTON
    monthcount_labelwidget = ttk.Label(statsmenu_frame, text='Components Count\n'
                                                             'by month (with pivoting): ',background='#DCDAD4')
    monthcount_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=monthcount_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_monthcount = ttk.Button(monthcount_frame, text='OK', command=lambda: count_per_month(),style='Button2.TButton')
    button_monthcount.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    monthcount_frame.grid(column=0, row=6, padx=10, pady=10,sticky='NW')

    # CREATE COUNT PER 13 WEEKS BUTTON
    yearcount_labelwidget = ttk.Label(statsmenu_frame, text='Components Count\n'
                                                            'for 13 weeks: ',background='#DCDAD4')
    yearcount_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=yearcount_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_yearcount = ttk.Button(yearcount_frame, text='OK', command=lambda: count_per_13_weeks(),style='Button2.TButton')
    button_yearcount.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    yearcount_frame.grid(column=0, row=7, padx=10, pady=10,sticky='NW')

    # CREATE OPEN GRAPHS TAB BUTTON
    open_graphs_tab_labelwidget = ttk.Label(statsmenu_frame, text='Open Graphs Tab: ',background='#DCDAD4')
    open_graphs_tab_frame = ttk.LabelFrame(statsmenu_frame,labelwidget=open_graphs_tab_labelwidget, borderwidth=4,style='Menu.TFrame')
    button_open_grapsh_tab = ttk.Button(open_graphs_tab_frame, text='OK', command=lambda: notebook_select(3),style='Button3.TButton')
    button_open_grapsh_tab.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    open_graphs_tab_frame.grid(column=0, row=8, padx=10, pady=10,sticky='NW')

    final_merged_frame = Frame(stats_frame, bg='#89AEBD')
    final_merged_frame.grid(column=1, row=0, padx=10, pady=10,sticky='N')


MODE PER MONTH FUNCTION

In [18]:
def mode_per_month():
    global final_merged_frame,user_activity,stats_frame, filtered

    # GROUP BY YEAR/MONTH AND COMPONENT INTO A SERIES
    modemon = filtered.groupby(['Year/Month'])['Component'].agg(pd.Series.mode)

    modemon_text = ttk.Label(final_merged_frame, text='Mode for Components by Month:', foreground='black',background='#89AEBD')
    modemon_text.pack(expand=False,fill='none')

    # CONVERT BACK TO DATAFRAME AND PLACE IN TAB
    modemon_df = pd.DataFrame(modemon)
    modemon_df.reset_index(inplace=True)
    load_table_view(modemon_df,final_merged_frame,7)

MODE PER 13 WEEKS FUNCTION

In [19]:
def mode_per_13_weeks():
    global final_merged_frame,user_activity,stats_frame,filtered

    # CONVERT TO SERIES
    mode13weeks = filtered['Component'].agg(pd.Series.mode)


    mode13weeks_text = ttk.Label(final_merged_frame, text='Mode for Components by 13 weeks:', foreground='black',background='#89AEBD')
    mode13weeks_text.pack(expand=False,fill='none')

    # CONVERT BACK TO DATAFRAME AND PLACE IN TAB
    mode13weeks_df = pd.DataFrame(mode13weeks)
    mode13weeks_df.reset_index(inplace=True)
    load_table_view(mode13weeks_df,final_merged_frame,8)

COUNT PER 13 WEEKS FUNCTION

In [20]:
def count_per_13_weeks():
    global final_merged_frame,user_activity,stats_frame,filtered


    count13weeks = filtered['Component'].value_counts()

    count13weeks_text = ttk.Label(final_merged_frame, text='Count by Component for 13 weeks:', foreground='black',background='#89AEBD')
    count13weeks_text.pack(expand=False,fill='none')

    count13weeks_df = pd.DataFrame(count13weeks)
    count13weeks_df.reset_index(inplace=True)
    load_table_view(count13weeks_df,final_merged_frame,9)

COUNT PER MONTH FUNCTION

In [21]:
def count_per_month():
    global final_merged_frame,user_activity,stats_frame,filtered

    # GROUP BY YEAR/MONTH AND COMPONENT, CALCULATE SUM AND UNSTACK
    countmon = filtered.groupby(['Year/Month','Component'])['Activity Count'].size().unstack('Component').fillna(0)

    countmon_text = ttk.Label(final_merged_frame, text='Count by Component per month:', foreground='black',background='#89AEBD')
    countmon_text.pack(expand=False,fill='none')

    # CONVERT BACK TO DATAFRAME AND PLACE IN TAB
    countmon_df = pd.DataFrame(countmon)
    countmon_df.reset_index(inplace=True)
    load_table_view(countmon_df,final_merged_frame,10)


MEAN PER MONTH FUNCTION

In [22]:
def mean_per_month():
    global final_merged_frame,user_activity,stats_frame,filtered

    # GROUP BY YEAR/MONTH AND COMPONENT, CALCULATE SUM AND UNSTACK
    meanmon = round(filtered.groupby(['Year/Month','Component'])['Activity Count'].mean().unstack('Component').fillna(0),2)
    meanmon_df = pd.DataFrame(meanmon)
    meanmon_df.reset_index(inplace=True)

    # CONVERT BACK TO DATAFRAME AND PLACE IN TAB
    meanmon_text = ttk.Label(final_merged_frame, text='Mean by Component per month:', foreground='black',background='#89AEBD')
    meanmon_text.pack(expand=False,fill='none')

    load_table_view(meanmon_df,final_merged_frame,11)

MEAN PER 13 WEEKS FUNCTION

In [23]:
def mean_per_13_weeks():
    global final_merged_frame,user_activity,stats_frame,filtered

    # GROUP BY COMPONENT BASED ON ACTIVITY COUNT AND EXTRACT MEAN
    mean13weeks = round(filtered['Activity Count'].groupby(filtered['Component']).mean(),2)
    mean13weeks_df = pd.DataFrame(mean13weeks)
    mean13weeks_df.reset_index(inplace=True)

    mean13weeks_text = ttk.Label(final_merged_frame, text='Mean by Component for 13 weeks:', foreground='black',background='#89AEBD')
    mean13weeks_text.pack(expand=False,fill='none')

    load_table_view(mean13weeks_df,final_merged_frame,12)

MEDIAN PER 13 WEEKS FUNCTION

In [24]:
def median_per_13_weeks():
    global final_merged_frame,user_activity,stats_frame,filtered
    # GROUP BY COMPONENT BASED ON ACTIVITY COUNT AND EXTRACT MEDIAN
    median13weeks = round(filtered['Activity Count'].groupby(filtered['Component']).median(),2)
    median13weeks_df = pd.DataFrame(median13weeks)
    median13weeks_df.reset_index(inplace=True)

    median13weeks_text = ttk.Label(final_merged_frame, text='Median by Component for 13 weeks:', foreground='black',background='#89AEBD')
    median13weeks_text.pack(expand=False,fill='none')

    load_table_view(median13weeks_df,final_merged_frame,13)

MEDIAN PER MONTH FUNCTION

In [25]:
def median_per_month():
    global final_merged_frame,user_activity,stats_frame,filtered
    # GROUP BY YEAR/MONTH AND COMPONENT, EXTRACT MEDIAN AND UNSTACK
    medianmon = round(filtered.groupby(['Year/Month','Component'])['Activity Count'].median().unstack('Component').fillna(0),2)
    medianmon_df = pd.DataFrame(medianmon)
    medianmon_df.reset_index(inplace=True)

    medianmon_text = ttk.Label(final_merged_frame, text='Median by Component per Month', foreground='black',background='#89AEBD')
    medianmon_text.pack(expand=False,fill='none')

    load_table_view(medianmon_df,final_merged_frame,14)

OPEN GRAPHS TAB

In [26]:
def graphs_tab():
    global activity_text,component_text,user_text,activity_frame,component_frame,user_frame,activity_log,user_log,button_cleanup,button_merge,button_pivot,button_multi,here_frame,merge_frame, button_statistics, final_merged_frame,stats_frame, db_for_statistics_reset, filtered,graphs_frame,button_open_graphs,graphs_frame,graphs_in_frame

    # CREATE COPY OF DB FOR GRAPHING OPERATIONS
    db_for_graphs = activity_count_reset.copy()
    indexes = [db_for_graphs['User_ID'],db_for_graphs['Year/Month'],db_for_graphs['Component'],db_for_graphs['Activity Count'],db_for_graphs['Day']]
    multiindex = pd.MultiIndex.from_arrays(indexes,names=('User_ID','Year/Month','Component','Activity Count','Day'))

    db_for_graphs.index = multiindex
    db_for_graphs.drop(['User_ID','Year/Month','Component','Activity Count','Day'],axis=1,inplace=True)

    # FILTER COMPONENTS ACCORDING TO REQUIREMENTS
    filtered = db_for_graphs[user_activity.index.get_level_values('Component').isin(['Quiz', 'Lecture', 'Assignment', 'Attendance', 'Survey'])]
    filtered.reset_index(inplace=True)

    notebook.tab(3, state='normal')

    # CONFIGURE STYLE FOR GRAPH TAB
    style.configure('Menu.TFrame',background='#DCDAD4',highlightcolor='#DCDAD4',)
    style.configure('LabelFrame.Label',background='#DCDAD4')
    style.configure('Frame5.TFrame',background='#97F2F3',border=NONE)
    style.configure('Button3.TButton',background='#97F2F3',border=NONE)

    # CREATE INNER FRAME FOR GRAPH TAB
    graphs_frame = ttk.Frame(graphs)
    graphs_frame.configure(style='Frame5.TFrame')
    graphs_frame.columnconfigure(0, minsize=100)
    graphs_frame.columnconfigure(1, weight=1)
    graphs_frame.rowconfigure(0, minsize=100)
    graphs_frame.rowconfigure(1, weight=1)
    graphs_frame.pack(expand=True, fill=BOTH)

    # CREATE FRAME TO HOST BUTTONS
    graphsmenu_labelwidget = ttk.Label(graphs, text='Graphs:',background='#DCDAD4')
    graphsmenu_frame = ttk.LabelFrame(graphs_frame,labelwidget=graphsmenu_labelwidget, style="Menu.TFrame")
    graphsmenu_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    userquiz_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Quiz:',background='#DCDAD4')
    userquiz_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=userquiz_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_userquiz = ttk.Button(userquiz_frame, text='OK', command=lambda: graph('Quiz'),style='Button3.TButton')
    button_userquiz.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    userquiz_frame.grid(column=0, row=0, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    userlecture_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Lecture:',background='#DCDAD4')
    userlecture_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=userlecture_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_userlecture = ttk.Button(userlecture_frame, text='OK', command=lambda: graph('Lecture'),style='Button3.TButton')
    button_userlecture.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    userlecture_frame.grid(column=0, row=1, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    userassignment_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Assignment:',background='#DCDAD4')
    userassignment_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=userassignment_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_userassignment = ttk.Button(userassignment_frame, text='OK', command=lambda: graph('Assignment'),style='Button3.TButton')
    button_userassignment.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    userassignment_frame.grid(column=0, row=2, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    userbook_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Book:',background='#DCDAD4')
    userbook_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=userbook_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_userbook = ttk.Button(userbook_frame, text='OK', command=lambda: graph('Book'),style='Button3.TButton')
    button_userbook.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    userbook_frame.grid(column=0, row=3, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    usercourse_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Course:',background='#DCDAD4')
    usercourse_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=usercourse_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_usercourse = ttk.Button(usercourse_frame, text='OK', command=lambda: graph('Course'),style='Button3.TButton')
    button_usercourse.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    usercourse_frame.grid(column=0, row=4, padx=10, pady=10,sticky='NW')

    # CREATE BUTTON
    userproject_labelwidget = ttk.Label(graphsmenu_frame, text='Correlation between\n'
                                                            'Users and Project:',background='#DCDAD4')
    userproject_frame = ttk.LabelFrame(graphsmenu_frame,labelwidget=userproject_labelwidget,  borderwidth=4,style='Menu.TFrame')
    button_userproject = ttk.Button(userproject_frame, text='OK', command=lambda: graph('Project'),style='Button3.TButton')
    button_userproject.grid(column=0, row=0, padx=10, pady=10,sticky='w')
    userproject_frame.grid(column=0, row=5, padx=10, pady=10,sticky='NW')


    graphs_in_frame = ttk.Frame(graphs_frame,style='Frame5.TFrame')
    graphs_in_frame.grid(column=1, row=0, padx=10, pady=10,sticky='NW')

    



RENDER GRAPH ON GRAPHS TAB

In [27]:
def graph(subject):
    global graphs_frame

    # FILTER COMPONENTS ACCORDING TO REQUIREMENTS
    filtered_graph = user_activity[user_activity.index.get_level_values('Component').isin(['Assignment', 'Quiz', 'Lecture', 'Book', 'Project', 'Course'])]
    # GROUP BY USER_ID AND COMPONENT, CALCULATE SUM AND UNSTACK
    filtered_graph_by_user = filtered_graph.groupby(['User_ID','Component']).size().unstack('Component').fillna(0)
    filtered_graph_by_user.reset_index(inplace=True)

    # DEFINE AND CONFIGURE PLOT
    fig, ax = plt.subplots(figsize=(10, 5),squeeze=True)
    y_data = str(subject)
    x_labels = filtered_graph_by_user['User_ID'].astype(str).tolist()
    x_ticks = filtered_graph_by_user['User_ID'].tolist()
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([''] * len(x_ticks))

    plt.title('Correlation between Users and ' + str(subject))
    plt.xlabel('User_ID', fontsize=10,labelpad=20)
    plt.ylabel(str(subject))

    # MANIPULATE LABELS IN ORDER NOT TO TRUNCATE
    label_interval = 10
    y_label_position = -50
    for i, label in enumerate(x_labels[::label_interval]):
        ax.text(x_ticks[i * label_interval], y_label_position, label, ha='center', rotation=90, fontsize=8)

    # CREATE SCATTER GRAPH
    ax.scatter(filtered_graph_by_user['User_ID'], filtered_graph_by_user[y_data], s=5)

    # PLOT GRAPH INTO FIGURE
    here_canvas = FigureCanvasTkAgg(fig, master = graphs_in_frame)
    here_canvas.draw()
    here_canvas.get_tk_widget().pack(pady=3)



GENERATE VISUAL TABLES FROM DATAFRAMES

In [28]:
def load_table_view(db,this_dbs_frame,db_number):

    style.map("Treeview.Heading",
   foreground=[('!active', '#FFFFFF'),('pressed', '#3e3e42'), ('active', '#3e3e42')],
    background=[ ('!active','#3e3e42'),('pressed', 'light gray'), ('active', 'light gray')]
    )

    # CREATE FRAME TO BE PLACED IN TAB THAT WAS PASSED BY REFERENCE
    sheet = ttk.Treeview(this_dbs_frame,selectmode="extended", height=20)
    clear_data(sheet)

    # CREATE SCROLLBAR
    # treescrolly = ttk.Scrollbar(sheet, orient="vertical", command=sheet.yview,)
    # # CONFIGURE SCROLLBAR
    # sheet.configure(yscrollcommand=treescrolly.set)
    # # PACK SCROLLBAR INTO FRAME
    # treescrolly.pack(side="right", fill="y")
    # PACK FRAME INTO TAB
    sheet.pack(expand=False, fill='none', anchor='e', padx=30, pady=10)


    # BIND SCROLLBAR TO EVENT ALLOWING IT TO CONTROL THE SCROLLING OF THE TREEVIEW
    # frame.bind("<Configure>", lambda event, sheet=sheet: on_frame_configure(this_dbs_frame))

    sheet["column"] = list(db.columns)
    # CONFIGURE MULTIPLE TABLES FOR THE PROJECT
    if db_number == 1:
        sheet.column(1, width=100, stretch=tk.NO)
        sheet.column(2, width=100, stretch=tk.NO)
    elif db_number == 3:
        sheet.column(1, width=150, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
    elif db_number == 4:
        sheet.column(1, width=100, stretch=tk.NO)
        sheet.column(2, width=100, stretch=tk.NO)
    elif db_number == 5:
        sheet.column(0, width=60, stretch=tk.NO)
        sheet.column(1, width=80, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
        sheet.column(3, width=80, stretch=tk.NO)
        sheet.column(4, width=80, stretch=tk.NO)
    elif db_number == 6:
        sheet.column(0, width=60, stretch=tk.NO)
        sheet.column(1, width=80, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
        sheet.column(3, width=100, stretch=tk.NO)
        sheet.column(4, width=40, stretch=tk.NO)
        sheet.column(5, width=47, stretch=tk.NO)
        sheet.column(6, width=60, stretch=tk.NO)
        sheet.column(7, width=150, stretch=tk.NO)
    elif db_number == 7:
        sheet['height'] = 5
    elif db_number == 8:
        sheet['height'] = 1
    elif db_number == 9:
        sheet['height'] = 5
    elif db_number == 10:
        sheet['height'] = 5
        sheet.column(0, width=80, stretch=tk.NO)
        sheet.column(1, width=80, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
        sheet.column(3, width=80, stretch=tk.NO)
        sheet.column(4, width=60, stretch=tk.NO)
        sheet.column(5, width=60, stretch=tk.NO)
    elif db_number == 11:
        sheet['height'] = 5
        sheet.column(0, width=80, stretch=tk.NO)
        sheet.column(1, width=80, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
        sheet.column(3, width=80, stretch=tk.NO)
        sheet.column(4, width=60, stretch=tk.NO)
        sheet.column(5, width=60, stretch=tk.NO)
    elif db_number == 12:
        sheet['height'] = 5
    elif db_number == 13:
        sheet['height'] = 5
    elif db_number == 14:
        sheet['height'] = 5
        sheet.column(0, width=80, stretch=tk.NO)
        sheet.column(1, width=80, stretch=tk.NO)
        sheet.column(2, width=80, stretch=tk.NO)
        sheet.column(3, width=80, stretch=tk.NO)
        sheet.column(4, width=60, stretch=tk.NO)
        sheet.column(5, width=60, stretch=tk.NO)

    # POPULATE HEADINGS AND COLUMNS WITH DATA
    sheet["show"] = "headings"
    for column in sheet["column"]:
        sheet.heading(column, text=column)
    df_rows = db.to_numpy().tolist()
    for row in df_rows:
        sheet.insert("", "end", values=row)


    return None

CLEAR TABLE BEFORE GENERATING A NEW ONE

In [29]:
def clear_data(etreeview):
    etreeview.delete(*etreeview.get_children())
    return None

SAVE JSON FILE

In [30]:
def save_file(dataset):
    global activity_log,component_codes,user_log
    dataset = int(dataset)

    if dataset == 1:
        f = filedialog.asksaveasfilename(defaultextension=".json")
        with open(f, "w", encoding="utf-8") as file:
            print(activity_log)
            js.dump(activity_log.to_json(), file, indent=4, ensure_ascii=False)
            messagebox.showinfo("Success","File Successfully saved")
    elif dataset == 2:
        f = filedialog.asksaveasfilename(defaultextension=".json")
        with open(f, "w", encoding="utf-8") as file:
            js.dump(component_codes.to_json(), file, indent=4, ensure_ascii=False)
            messagebox.showinfo("Success","File Successfully saved")
    elif dataset == 3:
        f = filedialog.asksaveasfilename(defaultextension=".json")

        with open(f, "w", encoding="utf-8") as file:
            js.dump(user_log.to_json(), file, indent=4, ensure_ascii=False)
            messagebox.showinfo("Success","File Successfully saved")
    else:
        messagebox.showerror("Error","Please select a dataset to save")
    button_save['state'] = 'normal'

END PROGRAM

In [31]:
def end_app():
    root.destroy()
    root.quit()

SCROLLBAR CONTROLLER

In [32]:
def on_frame_configure(scroll_frame):
    scroll_frame.configure(scrollregion=scroll_frame.bbox("all"))

GUI CREATOR AND MANAGER

In [ ]:
# CONFIGURE STYLES FOR WIDGETS
style.theme_use("clam")
style.configure('TLabelframe',background='#FFC0CB',bordercolor='#3e3e42',highlightbackground='#FFC0CB',borderwidth=1)
style.configure('TLabelframe.Label',foreground='#3e3e42',background='#FFC0CB')
style.configure(root,font='AppleGothicBlack',fontcolor='black',background='#3e3e42',foreground='#000000',borderwidth=2,bg='#3e3e42')
style.configure("TFrame",background="#dcdad6",foreground="#000000",borderwidth=4,relief="raised",fontbackground='white')
style.configure('TButton',background='#FEDFE4')

def donothing():
   filewin = Toplevel(root)
   button = Button(filewin, text="Do nothing button")
   button.pack()

menubar = Menu(root)
filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="Open CSV File", command=lambda: open_file())
save_menu = Menu(filemenu, tearoff=0)
save_menu.add_command(label="Save Activity Log", command=lambda: save_file(1),state='disabled')
save_menu.add_command(label="Save Component Codes", command=lambda: save_file(2),state='disabled')
save_menu.add_command(label="Save User Log", command=lambda: save_file(3),state='disabled')
filemenu.add_cascade(label="Save as JSON", menu=save_menu)
backup_menu = Menu(filemenu, tearoff=0)
backup_menu.add_command(label='BackUp Activity Log', command=lambda: save_to_mongo(1),state='disabled')
backup_menu.add_command(label='BackUp Component Codes', command=lambda: save_to_mongo(2),state='disabled')
backup_menu.add_command(label='BackUp User Log', command=lambda: save_to_mongo(3),state='disabled')
filemenu.add_cascade(label="BackUp to Mongo Server", menu=backup_menu)
filemenu.add_command(label='Open Clean Up',command=lambda: notebook_select(1),state='disabled')

filemenu.add_separator()
filemenu.add_command(label="Exit", command=lambda: end_app())
menubar.add_cascade(label="File", menu=filemenu)

dwmenu = Menu(menubar, tearoff=0)
dwmenu.add_command(label="Clean Up", command=lambda: cleanup(),state='disabled')
dwmenu.add_command(label="Merge", command=lambda: merge(),state='disabled')
dwmenu.add_command(label="Activity Count", command=lambda: activity_count(),state='disabled')
dwmenu.add_command(label="Open Statistics", command=lambda: notebook_select(2),state='disabled')
dwmenu.add_command(label="Open Graphs", command=lambda: notebook_select(3),state='disabled')
menubar.add_cascade(label="Data Wrangling", menu=dwmenu)

statsmenu = Menu(menubar, tearoff=0)
statsmenu.add_command(label="Count by Month", command=lambda: count_per_month(),state='disabled')
statsmenu.add_command(label="Count per 13 Weeks", command=lambda: count_per_13_weeks(),state='disabled')
statsmenu.add_command(label="Mean per Month", command=lambda: mean_per_month(),state='disabled')
statsmenu.add_command(label="Mean per 13 Weeks", command=lambda: mean_per_13_weeks(),state='disabled')
statsmenu.add_command(label="Median per Month", command=lambda: median_per_month(),state='disabled')
statsmenu.add_command(label="Median per 13 Weeks", command=lambda: median_per_13_weeks(),state='disabled')
statsmenu.add_command(label="Mode per Month", command=lambda: mode_per_month(),state='disabled')
statsmenu.add_command(label="Mode per 13 Weeks", command=lambda: mode_per_13_weeks(),state='disabled')
statsmenu.add_command(label="Open Statistics", command=lambda: notebook_select(2),state='disabled')
statsmenu.add_command(label="Open Graphs", command=lambda: notebook_select(3),state='disabled')
menubar.add_cascade(label="Statistics", menu=statsmenu)

graphsmenu = Menu(menubar, tearoff=0)
graphsmenu.add_command(label="Users and Assignment", command=lambda: graph('Assignment'),state='disabled')
graphsmenu.add_command(label="Users and Quiz", command=lambda: graph('Quiz'),state='disabled')
graphsmenu.add_command(label="Users and Lecture", command=lambda: graph('Lecture'),state='disabled')
graphsmenu.add_command(label="Users and Book", command=lambda: graph('Book'),state='disabled')
graphsmenu.add_command(label="Users and Project", command=lambda: graph('Project'),state='disabled')
graphsmenu.add_command(label="Users and Course", command=lambda: graph('Course'),state='disabled')
graphsmenu.add_command(label="Open Statistics", command=lambda: notebook_select(2),state='disabled')
graphsmenu.add_command(label="Open Graphs", command=lambda: notebook_select(3),state='disabled')
menubar.add_cascade(label="Graphs", menu=graphsmenu)

# CREATE CANVAS FOR SCROLLBAR
canvas = tk.Canvas(root, borderwidth=0, background="#3e3e42",width=1100,height=720)

# CREATE FRAME IN CANVAS TO HOST NOTEBOOK
frame = tk.Frame(canvas, background="#3e3e42",width=1080,height=720)

# CREATE NOTEBOOK IN FRAME AND ADD TABS
notebook = ttk.Notebook(frame,padding=10,width=1080,height=5000)
notebook.pack(expand=False)

# ADD FILE MANAGEMENT TAB TO NOTEBOOK
file_management = tk.LabelFrame(notebook, width=1080, height=700, background='#FFC0CB',borderwidth=1,relief='raised')
notebook.add(file_management, text="File Management")
# ADD DATASET MANAGEMENT TAB TO NOTEBOOK
dataset_management = tk.LabelFrame(notebook, width=1080, height=700, background='#F1CDB0',borderwidth=1,relief='raised')
notebook.add(dataset_management, text="Data Wrangling")
# START TAB DISABLED
notebook.tab(1,state="disabled")
# ADD STATISTICS TAB TO NOTEBOOK
statistics = tk.LabelFrame(notebook, width=1080, height=700, background='#89AEBD',borderwidth=1,relief='raised')
notebook.add(statistics, text="Statistics")
# START TAB DISABLED
notebook.tab(2,state="disabled")
# ADD GRAPHS TAB TO NOTEBOOK
graphs = tk.LabelFrame(notebook, width=1080, height=700, background='#97F2F3',borderwidth=1,relief='raised')
notebook.add(graphs, text="Graphs")
# START TAB DISABLED
notebook.tab(3,state="disabled")

# CREATE SCROLLBAR
vsb = tk.Scrollbar(root, orient="vertical", command=canvas.yview)

# CONFIGURE SCROLLBAR AT CANVAS
canvas.configure(yscrollcommand=vsb.set)

# PACK SCROLLBAR INTO CANVAS
vsb.pack(side="right", fill="y")

# LOAD FILE MANAGEMENT TAB
file_managent_tab()

# PACK CANVAS INTO ROOT WINDOW
canvas.pack(side="left", fill="both", expand=True)
# CREATE CANVAS FOR SCROLLBAR
canvas.create_window((1080,720), window=frame, anchor="nw")
# BIND SCROLLBAR TO EVENT
frame.bind("<Configure>", lambda event, canvas=canvas: on_frame_configure(canvas))


root.config(menu=menubar)

root.mainloop()